In [1]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import os

In [2]:
def pred(model, img):
    results = model.predict(img, conf=0.5, save=False, show=False)
    predicted = img.copy()

    boxes_lst = []
    
    for result in results:
        boxes = result.boxes
        for box in boxes:
            x_min, y_min, x_max, y_max = map(int, box.xyxy[0])
            boxes_lst.append((x_min, y_min, x_max, y_max))
            cv2.rectangle(predicted, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
    
            confidence = box.conf[0]
            class_id = int(box.cls[0])
            class_name = model.names.get(class_id, f'class_{class_id}')
            label = f'{class_name}: {confidence:.2f}'
            text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
            y = y_min - 10 if y_min - 10 > 10 else y_min + text_size[1] + 10
            cv2.putText(predicted, label, (x_min, y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
            
            
    return predicted, boxes_lst

### Trying Base YOLOv11

In [8]:
imgs_dir = 'datasets/test/images/'
fns = os.listdir(imgs_dir)
paths = [imgs_dir + fn for fn in fns]

model = YOLO('yolo11n.pt')
base_preds = []
for path in paths:
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    pred_img, _ = pred(model, img)
    base_preds.append(pred_img)


0: 640x640 1 person, 1 car, 8.0ms
Speed: 3.4ms preprocess, 8.0ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 train, 9.2ms
Speed: 2.0ms preprocess, 9.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 640x640 1 person, 1 truck, 1 horse, 1 clock, 8.6ms
Speed: 1.5ms preprocess, 8.6ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 8.7ms
Speed: 1.9ms preprocess, 8.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 640x640 (no detections), 8.5ms
Speed: 1.5ms preprocess, 8.5ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 8.0ms
Speed: 1.4ms preprocess, 8.0ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.9ms
Speed: 1.5ms preprocess, 7.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 8.7ms
Speed: 1.9ms preprocess, 8.7ms infer

In [9]:
out_dir = 'datasets/test/base_results/'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
    
out_paths = [out_dir + fn for fn in fns]
print(len(out_paths))
for i, img in enumerate(base_preds):
    cv2.imwrite(out_paths[i], img)

72


### Train

In [6]:
data_yaml_path = 'datasets/train.yaml'
model = YOLO('yolo11n.pt')
results = model.train(
    data=data_yaml_path, 
    epochs=100, 
    batch=16, 
    imgsz=640, 
    pretrained=True, 
    name='finetuned_yolov11_combined_640_augmented_2cls'
)

New https://pypi.org/project/ultralytics/8.3.130 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.127 🚀 Python-3.10.16 torch-2.6.0+cu124 CUDA:0 (NVIDIA A40, 45403MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets/train.yaml, degrees=0.0, deterministic=True, device=cuda:0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=finetuned_yolov11_combined_640_augmented_2cls, nbs=64, 

train: Scanning /home/ericzc2/project/AImbot/datasets/train/labels.cache... 1802


val: Fast image access ✅ (ping: 0.2±0.0 ms, read: 346.6±149.9 MB/s, size: 249.5 KB)


val: Scanning /home/ericzc2/project/AImbot/datasets/valid/labels.cache... 167 im


Plotting labels to runs/detect/finetuned_yolov11_combined_640_augmented_2cls/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/finetuned_yolov11_combined_640_augmented_2cls
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100      2.88G     0.9897     0.5802     0.9227         15        640: 1
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all        167        269      0.928       0.78      0.872      0.591



100 epochs completed in 0.477 hours.
Optimizer stripped from runs/detect/finetuned_yolov11_combined_640_augmented_2cls/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/finetuned_yolov11_combined_640_augmented_2cls/weights/best.pt, 5.5MB

Validating runs/detect/finetuned_yolov11_combined_640_augmented_2cls/weights/best.pt...
Ultralytics 8.3.127 🚀 Python-3.10.16 torch-2.6.0+cu124 CUDA:0 (NVIDIA A40, 45403MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  m


                   all        167        269      0.912      0.795      0.881      0.598
                person         85        124      0.968      0.782      0.883      0.602
               vehicle         88        145      0.855      0.807      0.879      0.593
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/finetuned_yolov11_combined_640_augmented_2cls


### Test

In [10]:
imgs_dir = 'datasets/test/images/'
fns = os.listdir(imgs_dir)
paths = [imgs_dir + fn for fn in fns]

trained_model = YOLO('runs/detect/finetuned_yolov11_combined_640_augmented_2cls/weights/last.pt')
trained_preds = []
trained_boxes = []
for path in paths:
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    pred_img, boxes_lst = pred(trained_model, img)
    trained_preds.append(pred_img)
    trained_boxes.append(boxes_lst)


0: 640x640 1 person, 7.8ms
Speed: 1.8ms preprocess, 7.8ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 vehicles, 8.3ms
Speed: 1.7ms preprocess, 8.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 640x640 1 person, 8.2ms
Speed: 1.3ms preprocess, 8.2ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 vehicle, 8.2ms
Speed: 1.8ms preprocess, 8.2ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 640x640 (no detections), 8.0ms
Speed: 1.5ms preprocess, 8.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.5ms
Speed: 1.3ms preprocess, 7.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 7.5ms
Speed: 1.2ms preprocess, 7.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 vehicle, 8.3ms
Speed: 2.0ms preprocess, 8.3ms inference, 1.2ms postprocess per image at shape (1, 3, 

In [13]:
out_dir = 'datasets/test/trained_results/'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
    
out_paths = [out_dir + fn for fn in fns]
for i, img in enumerate(trained_preds):
    cv2.imwrite(out_paths[i], img)

In [14]:
boxes_dir = 'datasets/test/trained_boxes/'
if not os.path.exists(boxes_dir):
    os.mkdir(boxes_dir)

for i, boxes in enumerate(trained_boxes):
    with open(boxes_dir + fns[i] + '.txt', 'w') as f:
        for b in boxes:
            b = str(b)[1:-1]
            f.write(f'{b}\n')